# Welcome to Advanced Software Engineering
## Analyzing Version Control System Data

![](https://comidoc.net/static/assets/thumbs/480/webp/2267906_505c.webp)

## Learning Objectives

After this session (+ exercises) the student will be able to:

  * Understand how research analyzing VCS histories collects the underlying data.
  * Export the history from Git repositories.
  * Apply scripts and programs in various languages to clean and pre-process the exported data.
  * Apply scripts and programs in various languages to analyze Git VCS data with respect to certain hypotheses/research questions.
  * Interpret the analysis results to either better understand current practices or to suggest actionable changes of current practices in software engineering teams.
  
-------------------------------------

# What are we doing today?

## Motivation

You all read the paper C. Bird et al. [_"Don't Touch My Code! Examining the Effects of Ownership on Software Quality"_]( https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/bird2011dtm.pdf)

In [ ]:
%%bash
curl -o bird2011dtm.pdf https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/bird2011dtm.pdf

In [ ]:
from IPython.display import IFrame


IFrame('./bird2011dtm.pdf', width='100%', height=800)

### What did you take away from the paper?


  * 
  * 
  



### Some paragraphs that I think are important...

  > "How much does ownership affect quality?"
    
  > Ownership is a general term used to describe whether one person has responsibility for a software component, or if there is no one clearly responsible developer. Within Microsoft, we have found that when more people work on a binary, it has more failures.
  
  > Interestingly, unlike some aspects of software which are known to be related to defects such as dependency complexity, or size, ownership is something that can be deliberately changed by modifying processes and policies. Thus, the answer to the question: “How much does ownership affect quality?” is important as it is actionable. Managers and team leads can make better decisions about how to govern a project by knowing the answer. 
  
   




  > We require several types of data. The most important data are the commit histories and software failures. Software repositories record the contents of every change made to a piece of software, along with the change author, the time of change, and an associated log message that may be indicative of the type of change (e.g. introducing a feature, or fixing a bug). We collected the number of changes made by each developer to each source file and used a mapping of source files to binaries in order to determine the number of changes made by each developer to each binary. Although the source code management system uses branches heavily, we only recorded changes from developers that were edits to the source code. Branching operations (e.g. branching and merging) were not counted as changes.

### After reading the paper, do you know how the researchers collected the data on which they base their analysis?

The big issue is now:
<p style="padding:6px; color: grey; background-color: white; border: orange 2px solid">"Alright, I can read the article and understand the experiment setup and results, but how would I run a similar experiment in my organization?"</p>

# What are we doing today?

## What are we really doing now?

We will -on a running example- perform a set of analyses of data from Git, a particular version control system (VCS). Everything in this lecture should be -with minor adaptations- applicable to other VCS, such as Mercurial, Microsoft TFS, etc.

You will learn how to:

  * Export the history from Git repositories.
  * Apply scripts and programs in various languages to clean and pre-process the exported data.
  * Apply scripts and programs in various languages to analyze Git VCS data with respect to certain hypotheses/research questions.
  * Interpret the analysis results to either better understand current practices or to suggest actionable changes of current practices in software engineering teams.
  
## Working together now

As you have various operating systems installed on your computers and since I cannot support all of these in such a short time, we will work together on a remotely hosted virtual machine Linux environment.

Navigate to: https://github.com/HelgeCPH/2019_ase_behavioural_analysis and press the button that says `launch binder`. In case you have a Linux or Unix installed on your computer you can run everything on your local machine right away.

After waiting a short while click on the file called `Lecture notes.ipynb` now you should see a similar screen to mine.

--------------------------------------------------------


# Getting the VCS Data

In the lecture we use [Apache Airflow](https://airflow.apache.org) as an example software system.

  > Airflow is a platform to programmatically author, schedule and monitor workflows.

The project is accessible on Github: https://github.com/apache/airflow. That is, we make use of the Git VCS during this lecture.

In [ ]:
%%bash
git clone https://github.com/apache/airflow.git

In [ ]:
%%bash
ls -ltrha airflow/

## Moving back in time 

We want to all work on the same "view" of the software so let's go back to the state of the repository on Monday Sept. 21rd.

In [ ]:
%%bash
cd airflow
git checkout $(git rev-list -n 1 --before="2020-09-21" master)

If you want to perform an analyzes for a certain period only, you can add the `--after` switch too.

## The revisions of a software system, organizational history

```bash
git log
```

Note, you have to switch into the directory of the repository that you want to study.

In [ ]:
%%bash
cd airflow/
git log | head -30

### What do we see and how to read it?

```
commit 0161b5ea2b805d62a0317e5cab6f797b92c8abf1
Author: Satyasheel <mlgruby@users.noreply.github.com>
Date:   Sat Sep 26 15:40:28 2020 +0100

    Increasing type coverage for multiple provider (#11159)

commit 08dfd8cd00dae2d7aad53018af04428d933b1ceb
Author: Satyasheel <mlgruby@users.noreply.github.com>
Date:   Fri Sep 25 21:08:26 2020 +0100

    Increase Type coverage for IMAP provider (#11154)

commit ee90807ace5d8b34ebd1f58ed4d9c831836d5a4d
Author: Ash Berlin-Taylor <ash_github@firemirror.com>
Date:   Fri Sep 25 20:49:11 2020 +0100

    Massively speed up the query returned by TI.filter_for_tis (#11147)
    
    The previous query generated SQL like this:
    
    
    WHERE (task_id = ? AND dag_id = ? AND execution_date = ?) OR (task_id = ? AND dag_id = ? AND execution_date = ?)
    
    Which is fine for one or maybe even 100 TIs, but when testing DAGs at
    extreme size (over 21k tasks!) this query was taking for ever (162s on
    Postgres, 172s on MySQL 5.7)
    
    By changing this query to this
```

Note, we are looking on the master branch only at the moment. That is, you see only commit messages on that single branch (we will talk more about these in three weeks).

```bash
git log --all
```

```bash
git log --branches --remotes --tags --graph --oneline --decorate
```


### What are all these switches?

You can read the help of `git log` via:

```bash
git help log
```

All the `git log`s we looked at so far are meant for humans to read and interpret.

However, we want to automatically analyze the logs to infer information out of it that is hidden in the logs.

---------------------------------------------

# Collecting, Cleansing, and Pre-processing the Data

## Exporting the `git log` into a machine readable format

Read the `PRETTY FORMATS` section of the `git log` help for possible placeholders in the format string

In [ ]:
%%bash
cd airflow/
git log --all --pretty=format:'%s' > ../data/all_commit_msgs.txt

In [ ]:
%%bash
head data/all_commit_msgs.txt

In [ ]:
%%bash
cd airflow/
git log --all --pretty=format:'"%h","%s"' > ../data/all_commit_msgs.csv

In [ ]:
%%bash
head data/all_commit_msgs.csv

In [ ]:
%%bash
cd airflow/
git log --pretty=format:'"%h","%an","%ad"' \
    --date=short \
    --numstat > \
    ../data/airflow_evo.log

In [ ]:
%%bash
head -30 data/airflow_evo.log

Not completely suited to our needs yet. Let's convert it into a CSV file with one file per line.

In [ ]:
%%bash
python evo_log_to_csv.py data/airflow_evo.log

In [ ]:
%%bash
ls -ltrh data

In [ ]:
%%bash
head data/airflow_evo.log.csv

Not completely nice either, what are the following "weird" files?

In [ ]:
%%bash
grep "=>" data/airflow_evo.log.csv | head

We have to "clean up" commits in which files have been moved so that we have a view of the current repository contents and do not lose old commit information.


In [ ]:
%%bash
python repair_git_move.py data/airflow_evo.log.csv

In [ ]:
%%bash
ls -ltrh data

In [ ]:
%%bash
head data/airflow_evo.log_repaired.csv

In [ ]:
%%bash
grep "=>" data/airflow_evo.log_repaired.csv | head

------------------------------------------------



# Shell-based Analysis

## How many revisions do we have in the repository?

That is: How much does the software change?

### Why does it matter?

  > In general, process measures based on the change history are more useful in predicting fault rates than product metrics of the code: For instance, the number of times code has been changed is a better indication of how many faults it will contain than is its length
  >
  > [Graves et al. _Predicting fault incidence using software change history_](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.116.9414&rep=rep1&type=pdf)

In [ ]:
%%bash
cut -d, -f 1 data/airflow_evo.log_repaired.csv | head

In [ ]:
%%bash
tail -n +2 data/airflow_evo.log_repaired.csv | head

In [ ]:
%%bash
tail -n +2 data/airflow_evo.log_repaired.csv | cut -d, -f 1 | head

In [ ]:
%%bash
tail -n +2 data/airflow_evo.log_repaired.csv | cut -d, -f 1 | sort | uniq | wc -l

#### How many revisions of each file do we have?

For the twenty most often changed files, we could formulate a query as in the following:

In [ ]:
%%bash
tail -n +2 data/airflow_evo.log_repaired.csv | cut -d, -f 7 | sort | uniq -c | sort -nr | tail -n +2 | head -20

# Shell-based Analysis

## Who are the ten persons that contribute the most?

### Why does it matter?

  > Our conclusion: adding more programmers improves the chances that a FOSS project will be successful.
  >
  > Schweik et al. _Brooks' Versus Linus' Law: An Empirical Test of Open Source Projects_


  > ...files with changes from nine or more developers were 16 times more likely to have a vulnerability than files changed by fewer than nine developers, indicating that many developers changing code may have a detrimental effect on the system's security
  >
  > [_Secure Open Source Collaboration: An Empirical Study of Linus’ Law_](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.965.7992&rep=rep1&type=pdf)

In [ ]:
%%bash
tail -n +2 data/airflow_evo.log_repaired.csv | cut -d, -f 2 | sort | uniq -c | sort -n | tail -n10 | tac

Note, on MacOS and other Unixes you can let tail reverse directly and you do not have the `tac` command by default.

```bash
tail -n +2 data/airflow_evo.log_repaired.csv | cut -d, -f 2 | sort | uniq -c | sort -n | tail -r -n10
```

In [ ]:
%%bash
tail -n +2 data/airflow_evo.log_repaired.csv | cut -d, -f 2 | sort | uniq -c | sort -n | tail -r -n10

### Your turn!

  * How many contributors are there in total?
 
<!--tail -n +2 data/airflow_evo.log_repaired.csv | cut -d, -f 2 | sort | uniq | wc -l -->

In [ ]:
%%bash
tail -n +2 data/airflow_evo.log_repaired.csv | cut -d, -f 2 | ???

  * Who is contributing the least?

<!-- tail -n +2 data/airflow_evo.log_repaired.csv | cut -d, -f 2 | sort | uniq -c | sort -n -->

In [ ]:
%%bash
tail -n +2 data/airflow_evo.log_repaired.csv | cut -d, -f 2 | ???

  * Google-fu question: What is the median contribution value?
  
<!-- tail -n +2 data/airflow_evo.log_repaired.csv | cut -d, -f 2 | sort | uniq -c | cut -d '"' -f 1 | python -c"import sys,statistics;print(statistics.median([int(i) for i in sys.stdin.readlines()]))" -->

In [ ]:
%%bash
tail -n +2 data/airflow_evo.log_repaired.csv | cut -d, -f 2 | ???

# Scripted Analysis 1: 

## What are people doing?

We can analyze the commit messages to get a feeling for what people are doing.

### Why does it matter?

  > If we look at the commit cloud, we see that certain terms dominate. What you'll learn right now is by no means scientific, but it's a useful heuristic: the words that stand out tell you where you spend your time.
  >
  > A. Tornhill _Your Code as a Crime Scene_

In [ ]:
%%bash
cat data/all_commit_msgs.txt | head

In [ ]:
%%bash
grep "[Ff]ix" data/all_commit_msgs.txt | head

In [ ]:
%%bash
grep "[Ff]ix" data/all_commit_msgs.txt | wc -l

In [ ]:
%%bash
grep "[Ee]rror" data/all_commit_msgs.txt | wc -l

In [ ]:
%%bash
grep "[Bb]ug" data/all_commit_msgs.txt | wc -l

With a bit enhanced word natural-language processing (NLP) based word extraction, we can condense the following word frequencies.

In [ ]:
%%bash
python word_freq.py data/all_commit_msgs.txt nlp

Let's create a wordcloud for the data.

In [ ]:
%%bash
python wordcloud_gen.py data/all_commit_msgs.txt nlp

In [ ]:
%%bash
ls -ltrh out

![](out/all_commit_msgs.png)

# Scripted Analysis 2 

## Sentiment Analysis of Commit Messages

### Why does it matter?

Commit messages can give insight into the state of your team and likely also into the state of your software.

For example they can shed light on when your team members are "working best" and in which setup:

  > Our results show that projects developed in Java tend to have more negative commit comments, and that projects that have more distributed teams tend to have a higher positive polarity in their emotional content. Additionally, we found that commit comments written on Mondays tend to a more negative emotion.
  >
  > [E. Guzman et al. _"Sentiment Analysis of Commit Comments in GitHub: An Empirical Study"_](https://www.researchgate.net/profile/Emitza_Guzman/publication/266657943_Sentiment_analysis_of_commit_comments_in_GitHub_An_empirical_study/links/5b8305ba4585151fd134f10c/Sentiment-analysis-of-commit-comments-in-GitHub-An-empirical-study.pdf)

In [ ]:
%%bash
python commit_sentiments.py data/all_commit_msgs.csv > data/all_commit_sentiment_msgs.csv

In [ ]:
%%bash
ls -ltrh data

In [ ]:
%%bash
head data/all_commit_sentiment_msgs.csv

In [ ]:
import pandas as pd


df = pd.read_csv('data/all_commit_sentiment_msgs.csv')
df.head()

#### Which are the commits containing negative commit messages?

In [ ]:
df[df.polarity < -0.5]

#### Which are the commits containing positive commit messages?

In [ ]:
df[df.polarity > 0.5]

**Possible Project:**

Train an NLP sentiment analysis model on a properly annotated commit history to get better results.

#### Which is the commit with the most negative commit message?

In [ ]:
most_negative = df[df.polarity == df.polarity.min()]
print(most_negative.msg.values)
most_negative

## What is the corresponding commit?

Now we are crossing information from two datasets.

In [ ]:
complete_df = pd.read_csv('data/airflow_evo.log_repaired.csv')
complete_df[complete_df.hash == most_negative.index[0]]

# Scripted Analysis 2 

## A Project's Bus-factor


One of the early mentions of the term in a real project [_"If Guido was hit by a bus?"_](https://legacy.python.org/search/hypermail/python-1994q2/1040.html)


### Why does it matter?

  > ...the number of people on your team that have to be hit by a truck (or quit) before the project is in serious trouble...
  >
  > L. Williams and R. Kessler, Pair Programming Illuminated. Wesley, 2003.
Addison

In [ ]:
complete_df.head()

Let's start with computing code ownership. There are different possibilities to define and measure code ownership:

a)
  > One measure of ownership is  how  much  of  the  development  activity  for  a  component comes from one developer.  If one developer makes 80% of the changes to a component, then we say that the component has high ownership.  The other way that we measure ownership  is  by  determining  how  many  low-expertise  developers are working on a component.  If many developers are all making few changes to a component, then there are many non-experts working on the component and we label the component as having low ownership
  >
  > C. Bird et al. [_"Don't Touch My Code! Examining the Effects of Ownership on Software Quality"_]( https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/bird2011dtm.pdf)


  > Proportion of Ownership – The proportion of ownership  (or  simply  ownership)  of  a  contributor  for a particular component is the ratio of number of commits that the contributor has made relative to the total number of commits for that component.
  >
  > C. Bird et al. [_"Don't Touch My Code! Examining the Effects of Ownership on Software Quality"_]( https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/bird2011dtm.pdf)

b)

There are more elaborate measures for code ownership in the literature, for example:

  > $DOA = 3.293 + 1.098 ∗ FA + 0.164 ∗ DL − 0.321 ∗ ln(1 + AC )$
  >
  > The degree of authorship of a developer d in a file f depends on three factors: first authorship (FA), number of deliveries (DL), and number of acceptances (AC). If d is the author of f, FA is 1; otherwise it is 0; DL is the number of changes in f made by D; and AC is the number of changes in f made by other developers.
  > 
  > [G. Avelino et al. _What is the Truck Factor of Popular GitHub Applications? A First Assessment_](https://peerj.com/preprints/1233.pdf)
  
c) 

In *Your Code as a Crime Scene*, A. Tornhill suggests to compute a simple measure -which he calls it *knowledge ownership*- that counts the total amount of lines added through the history of a file. And additionally, per author the amounts of lines added by that author. Out of the two values he computes a the _knowledge ownership_ rate.



For the following example, we go for the latter definition and in our computation we just say that the author with the highest _knowledge ownership_ rate on a file "owns" that file.

In [ ]:
complete_df.head()

In [ ]:
fname = 'scripts/ci/_utils.sh'
complete_df[complete_df.current == fname]

In [ ]:
new_rows = []

for fname in set(complete_df.current.values):
    view = complete_df[complete_df.current == fname]
    sum_series = view.groupby(['author']).added.sum()
    view_df = sum_series.reset_index(name='sum_added')
    total_added = view_df.sum_added.sum()
    
    if total_added > 0:  # For binaries there are no lines counted
        view_df['owning_percent'] = view_df.sum_added / total_added
        owning_author = view_df.loc[view_df.owning_percent.idxmax()]
        new_rows.append((fname, owning_author.author, owning_author.sum_added,
                         total_added, owning_author.owning_percent))
    # All binaries are silently skipped in this report...

In [ ]:
owner_df = pd.DataFrame(new_rows, columns=['artifact', 'main_dev', 'added', 'total_added', 'owner_rate'])
owner_df.head()

In [ ]:
owner_freq_series = owner_df.groupby(['main_dev']).artifact.count()
owner_freq_series

In [ ]:
owner_freq_df = owner_freq_series.reset_index(name='owns_no_artifacts')
owner_freq_df.sort_values(by='owns_no_artifacts', inplace=True)
owner_freq_df

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt


fig = plt.figure(figsize=(12, 3))
plt.xticks(rotation=45)
plt.xticks(size=3)
plt.plot(owner_freq_df.main_dev, owner_freq_df.owns_no_artifacts)

Now, we can -similar to 
<a href="https://ieeexplore.ieee.org/stamp)/stamp.jsp?arnumber=7503718">G. Avelino et al. *A novel approach for estimating Truck Factors*</a>- start "removing" low-contributing authors from the dataset as long as still more than half of the files have an owner. The amount of remaining owners is the bus-factor of that project.

In [ ]:
no_artifacts = len(owner_df.artifact)
half_no_artifacts = no_artifacts // 2
count = 0

for owner, freq in owner_freq_df.values:
    no_artifacts -= freq
    if no_artifacts < half_no_artifacts:
        break
    else:
        count += 1

busfactor = len(owner_freq_df.main_dev) - count
print(f"The bus factor of Apache Airflow is: {busfactor}")

----------------------------------------------------------------


# Your turn!

![](http://giphygifs.s3.amazonaws.com/media/11M1k4fIwVqPF6/giphy.gif)

Chose one or more software systems that are under version control with Git.

Use the version history as done during class (if you want to in combination with the provided Python scripts) to analyze VCS history according to one of the small projects below.

Base your analysis on the file `<your_system>_evo.log.csv` that you can create similar to the examples above like:

```bash
git clone <url to>/<your_system>.git

cd <your_system>/
git log --pretty=format:'"%h","%an","%ad"' \
    --date=short \
    --numstat > \
    ../data/<your_system>_evo.log

python evo_log_to_csv.py data/<your_system>_evo.log
python repair_git_move.py data/<your_system>_evo.log.csv
```

You can implement your analysis in the languages and with technologies of your choice. (Likely your favorite scripting language comes in handy here.)

If you do not like the shell-based analysis that relies on Git logs, you may want to consider tools, such as, [PyDriller](https://github.com/ishepard/pydriller), which provide a more high-level API for many of the actions presented in this class.



I suggest that you could work on one of the following problems:


### A) Can you suggest a team structure for a project?

Persons who work often on the same artifacts should perhaps work physically together and not be distributed around the globe, see e.g., [M. Penta et al. _The Effect of Communication Overhead on Software Maintenance Project Staffing: a Search-Based Approach_](http://crest.cs.ucl.ac.uk/fileadmin/crest/sebasepaper/DiPentaHAQ07.pdf) and N. Nagappan et al. [_The Influence of Organizational Structure on Software Quality_](https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/tr-2008-11.pdf).

Write a script/program that suggests teams of size five to ten persons.

Find a suitable categorization for all persons that rarely contribute to a project.

Chapters 11-13 in A. Tornhill _Your Code as a Crime Scene_ will likely form a good basis for the task.


### B) Hotspot Analysis

Based on A. Tornhill _Your Code as a Crime Scene_ chapters 3-5, write a script/program that identifies and visualizes hotspots (often changing and complex artifacts)

You can make use of `cloc` (installed into this VM) to compute the size (proxy for complexity) of a file:

```bash
./cloc --csv --by-file airflow
```

Alternatively, you can write another small script that computes the whitespace complexity of a file, as in chapter 6


### C) Architectural Decay

Investigate potential architectural decay based on measures for temporal coupling chapters 7-8 (and 9-10) in A. Tornhill _Your Code as a Crime Scene_.

Write a script/program that suggests artifacts that seem to be prone to architectural decay when their temporal couples increase drastically over an analysis period.


### D) Reproduce Paper Results

Take the initial paper from https://peerj.com/preprints/1233.pdf and the final version <a href="https://ieeexplore.ieee.org/stamp)/stamp.jsp?arnumber=7503718">G. Avelino et al. _A novel approach for estimating Truck Factors_</a>. Download the given repositories, implement a computation of ownership as described in the papers, and finally compare how the bus-factors developed since the papers were published to now.

### E) Freestyle

Formulate a problem that you want to investigate based on the history of a VCS. An example for such a problem formulation could be:
  
  * Can we find a relation between sentiments in commit messages and complexity of the files in the commit?

Write a script/program that implements the analysis for the given problem and generates suitable result data or plots.



## Formalities

  * Chose a project now and work on it for the rest of today's session.
<!--  * I will be there for the session on Thursday to help with practicalities. -->
  * Prepare a small presentation of your results and findings for next Monday. We will start the session with a small presentation per group.


# I do not have Linux on my computer...

All of the scripts and advise in this material assumes that you are working on a Linux/Unix environment. In case you are on Windows, there are many different ways of setting up such an environment. You can find a guide on setting up virtual machines with -amongst others- Ubuntu Linux here: http://itu.dk/people/ropf/blog/vagrant_install.html. I recommend that you setup such an environment as you as software engineers should be comfortable and able to work in various environments depending on the needs of your future companies/clients.

For small analysis that do not require a lot of resources, you might want to fork and adapt this repository, so that you can run your code on mybinder.